# Spiritual AI Agent

This project is a **Spiritual AI Agent** built using [LangChain](https://python.langchain.com/), [Google Gemini](https://ai.google.dev/), and various web APIs. The agent provides empathetic, wise, and actionable spiritual guidance, including book recommendations, quotes, places, images, and videos tailored to the user's needs.

## Features

- **Intent Classification:** Understands the user's spiritual needs (e.g., emotional crisis, seeking knowledge, inspiration).
- **Book Recommendations:** Suggests relevant spiritual books using the Google Books API.
- **Spiritual Places:** Finds nearby temples, meditation centers, or yoga classes using Serper API.
- **Image Search:** Shows images of spiritual places.
- **YouTube Videos:** Recommends related talks, meditations, or discourses.
- **Inspirational Quotes:** Fetches motivational and spiritual quotes from ZenQuotes.
- **Web Search:** For additional spiritual resources.
- **Human Input Tool:** Asks clarifying questions when needed.
- **Summary Tool:** Gently summarizes the guidance, including links and resources.

## Setup

1. **Install Dependencies**

    ```bash
    pip install langchain-community langchain-google-genai youtube_search
    ```

2. **API Keys Required**
    - **Google AI API Key** (for Gemini)
    - **Serper API Key** (for web, places, and image search)

    The notebook will prompt you to enter these keys if not set in your environment.

3. **Run the Notebook**
    - Follow the cells in order, or use the provided agent executor to interact with the agent.

## Usage

- **Ask Spiritual Questions:**  
  The agent can help with finding peace, understanding spiritual concepts, recommending practices, or suggesting places and resources.
- **Example:**
  ```python
  response = agent_executor.invoke({
        "input": "Lately, I’ve been feeling lost and disconnected from my purpose. I want to find peace and reconnect with my spiritual side. Can you guide me?"
  })
  print(response["output"])
  ```

## Tools Integrated

- **Web_Search_Tool:** General spiritual web search.
- **Formatted_Places_Tool:** Human-readable list of spiritual places.
- **Formatted_Image_Tools:** Image links for places.
- **YouTube_Video_Search_Tool:** Spiritual video recommendations.
- **Spiritual_Books_Tool:** Book recommendations.
- **Spiritual_Quotes_Tool:** Inspirational quotes.
- **HumanInputTool:** Clarifies user intent or location.
- **Spiritual_Intent_Classifier:** Classifies user needs.
- **Summary_Tool:** Summarizes the response.

## Customization

- You can add or modify tools as needed.
- The prompt template can be adjusted for tone or flow.

## License

This project is for educational and research purposes. Please respect the terms of the APIs used.

---

*May your journey be peaceful and insightful!*

In [ ]:
%pip install langchain-community
from langchain_community.tools import Tool
from langchain_core.tools import tool

In [ ]:
%pip install -U langchain-google-genai

In [3]:
import getpass
import os
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import HarmCategory, HarmBlockThreshold

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    }
)

In [5]:
import getpass
import os
if "SERPER_API_KEY" not in os.environ:
    os.environ["SERPER_API_KEY"] = getpass.getpass("Enter your Serper AI API key: ")

In [6]:
#Web Search tool
from langchain_community.utilities import GoogleSerperAPIWrapper
search_general = GoogleSerperAPIWrapper()
search_tool=  Tool(
        name="Web_Search_Tool",
        func=search_general.run,
        description="useful when you need to search for any information on the web",
    )

In [7]:
#Search Places Tool
search_places = GoogleSerperAPIWrapper(type="places")
search_places_tool=Tool(
    name="Places_Search_Tool",
    func=search_places.results,
    description="useful when you need to search for places on the web",
)

In [8]:
#Search Images Tool
search_image = GoogleSerperAPIWrapper(type="images")
search_image_tool=  Tool(
        name="Image_Search_Tool",
        func=search_image.results,
        description="useful when you need to search for images on the web",
    )

In [ ]:
@tool
def formatted_places_tool(query: str) -> str:
    """Returns a human-readable list of places from Serper."""
    places = search_places.results(query)["places"]
    if not places:
        return "No places found."
    result = "📍 **Top Places:**\n"
    for p in places:
        result += f"- **{p.get('title')}** ({p.get('rating', 'N/A')}★)\n"
        result += f"  - Address: {p.get('address', '')}\n"
        if p.get("website"):
            result += f"  - Website: {p['website']}\n"
    return result
Formatted_Search_Places_Tool=Tool(name='Formatted_Places_Tool',func=formatted_places_tool,description='useful when you need to search for places on the web')
# print(Formatted_Search_Places_Tool)

In [10]:
@tool
def formatted_image_links_query(query: str) -> str:
    """Returns a human-readable list of image links from Serper."""
    news = search_image.results(query)["images"]
    if not news:
        return "No images found."
    result=""
    for p in news:
        result += f"- **{p.get('title')}**"
        result += f"  - Link: {p.get('imageUrl', '')}\n"
    return result
Formatted_Image_Links_Tool=Tool(name='Formatted_Image_Tools',func=formatted_image_links_query,description='useful when you need to search for images on the web')
# print(Formatted_Image_Links_Tool)

In [11]:
%pip install --upgrade --quiet  youtube_search

Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain_core.tools import Tool, tool
from langchain_community.tools import YouTubeSearchTool

# Instantiate the raw tool
raw_yt_tool = YouTubeSearchTool()

# Create safe wrapper tool
@tool
def youtube_video_search(query: str) -> str:
    """
    Search YouTube for spiritual videos, guided meditations, or discourses.
    Automatically returns 2 results for any input string.
    """
    try:
        formatted_input = f"{query}, 2"
        return raw_yt_tool.run(formatted_input)
    except Exception as e:
        return f"⚠️ YouTube search failed: {str(e)}"
YT_Video_Recommend_Tool = Tool(
    name="YouTube_Video_Search_Tool",
    func=youtube_video_search,
    description="Search YouTube for videos related to spirituality, healing, or meditation"
)


In [13]:
@tool
def generate_summary(input: str) -> str:
    """
    Combines all steps to create a user-readable summary.
    Include the image , video links.
    Input could be a JSON or compiled string from other tools.
    """
    return f"Summary:\n{input}\n(You can export this as a PDF using pdfkit or FPDF)"
Summary_Tool=Tool(name='Summary_Tool',func=generate_summary,description='Combines all steps to create a user-readable summary . Input could be a JSON or compiled string from other tools')

In [14]:
import requests
@tool
def get_spiritual_books(query: str) -> str:
    """Searches for spiritual books related to the user's interest."""
    url = f"https://www.googleapis.com/books/v1/volumes?q={query}"
    response = requests.get(url)
    data = response.json()
    if not data.get("items"):
        return "No books found for that topic."

    results = []
    for book in data["items"][:3]:  # limit to 3 recommendations
        info = book["volumeInfo"]
        title = info.get("title", "Unknown Title")
        authors = ", ".join(info.get("authors", []))
        link = info.get("infoLink", "")
        results.append(f"📘 *{title}* by {authors}\n🔗 [More Info]({link})")

    return "\n\n".join(results)
# print(get_spiritual_books("Meditation & Yoga"))

In [15]:
from langchain_core.tools import tool
import requests

@tool
def fetch_spiritual_quotes(_: str = "") -> list:
    """
    Fetches a list of spiritual/inspirational quotes from ZenQuotes API.
    Returns a list of formatted strings like: "Quote" — Author.
    Let the LLM decide which quote is best for the user's query.
    """
    try:
        response = requests.get("https://zenquotes.io/api/quotes")
        if response.status_code != 200:
            return ["⚠️ Failed to fetch quotes from ZenQuotes API."]

        quotes_data = response.json()
        quotes = []
        for quote in quotes_data[:10]:
            text = quote.get("q", "")
            author = quote.get("a", "Unknown")
            quotes.append(f"🧘 \"{text}\" — *{author}*")
        return quotes
    except Exception as e:
        return [f"⚠️ Error: {str(e)}"]
spiritual_quote_tool=Tool(name='Spiritual_Quotes_Tool',func=fetch_spiritual_quotes,description='useful for fetching quotes related to spirituality,motivation,success')
print(spiritual_quote_tool.name)

Spiritual_Quotes_Tool


In [16]:
@tool()
def spiritual_intent_classifier(user_input: str) -> str:
    """
    Classifies the user's spiritual need into one of the following categories:
    - 'emotional_crisis'
    - 'seeking_knowledge'
    - 'looking_for_practices'
    - 'travel_or_pilgrimage'
    - 'existential_question'
    - 'seeking_inspiration'
    - 'general_spiritual_exploration'

    The tool should be used before deciding which spiritual tool or path to follow.
    """
    # This tool returns a classification. LLM will decide based on text.
    return (
        "Please analyze the user's input and classify it into one of the spiritual intent categories listed."
    )

In [17]:
# Or you can directly instantiate the tool
# !pip install langchain-community
from langchain_community.tools import HumanInputRun
HumanInputTool = HumanInputRun(name='HumanInputTool')
print(HumanInputTool.name)

HumanInputTool


In [ ]:
tools=[
    search_tool,
    Formatted_Search_Places_Tool,
    Formatted_Image_Links_Tool,
    YT_Video_Recommend_Tool,
    Summary_Tool,
    get_spiritual_books,
    spiritual_quote_tool,
    HumanInputTool,
    spiritual_intent_classifier
]

In [19]:
from langchain.prompts import PromptTemplate
react_template = """
You are a helpful, wise, and compassionate spiritual assistant who provides empathetic guidance, teachings, and suggestions to support users on their spiritual journey.
If at any point the user's question is out of scope, say:
"I’m here to guide you on your spiritual path. For other types of questions, I kindly suggest using a more general assistant."

You have access to the following tools:
{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Follow this flow to generate your response:

1. 🧠 First, determine the user's **spiritual need or intent** using the `Spiritual_Intent_Classifier` tool, if you are not able to determine it then simply ask the user using the `HumanInputTool`.
2. 🪞 Then, offer a calm, **personalized reflection or perspective** on their situation in your own words.
3. 📚 Call the `Spiritual_Books_Tool` to recommend books that are relevant to their situation — or general suggestions on spirituality, yoga, or meditation.
4. 🧘‍♂️ Call the `HumanInputTool` to ask for their **location (city/state)**. Then use the `Formatted_Places_Tool` to recommend nearby **spiritual places** such as temples, yoga classes, meditation centers, or sacred sites. If no local places are found, suggest national or regional options.
Don't call this tool if the user's intent is to seek knowledge, because places is not relevant in that case
5. 🖼️ Use the `Formatted_Image_Tools` to show images of each of the place which you got in the previous step .
6. 🎥 Use the `YouTube_Video_Search_Tool` to recommend **videos or talks** related to the user’s situation or related to user's question to provide comfort and spiritual direction.
7. 🧘‍♀️ Use the `Spiritual_Quotes_Tool` to provide one or more motivational or transformative quotes. If no relevant quote is found then find one using the `Web_Search_Tool`. If the intent is about seeking knowledge then dont call this tool.
8. ✨ Use the `Summary_Tool` to **gently summarize** your full response into a peaceful and helpful final message,include the youtube video links , image links(if the tool is called) as a part of the summary.

You may use the `Web_Search_Tool` only for topics related to spirituality , yoga and meditation if you are unable to answer them.
🕊️ Throughout the conversation:
- Be empathetic, insightful, and spiritually grounded
- Cite tool observations where appropriate
- Use the `HumanInputTool` to clarify user intent or fix irrelevant input
- If a tool fails or returns nothing, respond calmly and fallback with insight

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""
# Formatted_Image_Tools
# Formatted_Places_Tool

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [20]:
from langchain.agents import create_react_agent, AgentExecutor

agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=False,
     max_iterations=15,   # Increase from default
    max_execution_time=120  # in seconds
)


In [101]:
response = agent_executor.invoke({
    "input": "Lately, I’ve been feeling lost and disconnected from my purpose. I want to find peace and reconnect with my spiritual side. Can you guide me?"
})
print(response["output"])




> Entering new AgentExecutor chain...
Okay, I understand you're feeling lost and disconnected and are seeking guidance to reconnect with your spiritual side and find peace. It's a common feeling, and a courageous step to seek guidance. Many people experience this sense of disconnection at some point in their lives. Remember that this feeling, while uncomfortable, can be a powerful catalyst for growth and self-discovery. It's like a gentle nudge from your soul, urging you to remember who you truly are.

First, I want to understand your needs better, so I'll use the `spiritual_intent_classifier` to determine your intent.
Action: spiritual_intent_classifier
Action Input: Lately, I’ve been feeling lost and disconnected from my purpose. I want to find peace and reconnect with my spiritual side. Can you guide me?Please analyze the user's input and classify it into one of the spiritual intent categories listed.Okay, the intent seems to be a mix of seeking inspiration and looking for practic

In [102]:
response2 = agent_executor.invoke({
    "input": "I want to understand what karma really means and how I can align my actions with my spiritual growth."
})
print(response2["output"])




> Entering new AgentExecutor chain...
Okay, I understand you're seeking to understand karma and align your actions with your spiritual growth. That's a beautiful intention. It's like learning to dance with the universe, moving in harmony with its rhythms.

Action: spiritual_intent_classifier
Action Input: I want to understand what karma really means and how I can align my actions with my spiritual growth.Please analyze the user's input and classify it into one of the spiritual intent categories listed.Okay, the user is seeking knowledge about karma and its relation to spiritual growth.

Action: spiritual_intent_classifier
Action Input: I want to understand what karma really means and how I can align my actions with my spiritual growth.Please analyze the user's input and classify it into one of the spiritual intent categories listed.Okay, the user is seeking knowledge about karma and its relation to spiritual growth.
Action: seeking_knowledge
Action Input: I want to understand what ka

In [103]:
#
response3 = agent_executor.invoke({
    "input": "Can you suggest daily meditation or breathing techniques to calm my mind and stay more focused?"
})
print(response3["output"])




> Entering new AgentExecutor chain...
Okay, I understand you're looking for daily meditation or breathing techniques to calm your mind and improve focus. It's wonderful that you're seeking ways to cultivate inner peace and clarity. Many people find that incorporating these practices into their daily routine can significantly enhance their well-being.

First, I'll use the `spiritual_intent_classifier` to better understand your needs.
Action: spiritual_intent_classifier
Action Input: Can you suggest daily meditation or breathing techniques to calm my mind and stay more focused?Please analyze the user's input and classify it into one of the spiritual intent categories listed.Okay, the intent seems to be about finding practices to calm the mind and improve focus. Now I will use the `get_spiritual_books` tool to find some relevant books.
Action: get_spiritual_books
Action Input: meditation and breathing techniques for calming mind and focus📘 *Do Breathe* by Michael Townsend Williams
🔗 [Mo

In [106]:
response5 = agent_executor.invoke({
    "input": "I am feeling depressed and can't focus on my life. It feels like there is no zeal for my life. My energy is very low these days. Please guide me"
})
print(response5["output"])



> Entering new AgentExecutor chain...
It sounds like you're going through a difficult time, feeling depressed, lacking focus, and experiencing low energy. It's important to acknowledge these feelings and be gentle with yourself. Remember that you're not alone, and many people experience similar challenges in their lives. Spirituality can offer solace, guidance, and tools to help you navigate these feelings and rediscover your inner strength and zeal for life.

I will start by identifying your intent using the `spiritual_intent_classifier` tool, and then I will suggest books, spiritual places, videos and quotes to help you.

Action: spiritual_intent_classifier
Action Input: I am feeling depressed and can't focus on my life. It feels like there is no zeal for my life. My energy is very low these days. Please guide mePlease analyze the user's input and classify it into one of the spiritual intent categories listed.Okay, the intent is classified as 'emotional_crisis'. It's understandable

In [104]:
# How do I fix a network issue on my Windows PC?
response4 = agent_executor.invoke({
    "input": "How do I fix a network issue on my Windows PC?"
})
print(response4["output"])




> Entering new AgentExecutor chain...
The user is asking about fixing a network issue on their Windows PC. This is a technical question and not related to spirituality.

Action: spiritual_intent_classifier
Action Input: How do I fix a network issue on my Windows PC?Please analyze the user's input and classify it into one of the spiritual intent categories listed.The user is asking about fixing a network issue on their Windows PC. This is a technical question and not related to spirituality.

Action: HumanInputTool
Action Input: I’m here to guide you on your spiritual path. For other types of questions, I kindly suggest using a more general assistant. Is there anything I can help you with on your spiritual journey?

I’m here to guide you on your spiritual path. For other types of questions, I kindly suggest using a more general assistant. Is there anything I can help you with on your spiritual journey?
no
noFinal Answer: I’m here to guide you on your spiritual path. For other types of